In [1]:
# DATA READ - REYNOLDS AND VELOCITY VALUES
import pandas as pd
import os
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#import data

file_name = 'fxReynoldsCFD_ANN.xlsx'
file_path = os.path.join(os.getcwd(), file_name)

ANNDF = pd.read_excel(file_path)

X = ANNDF.iloc[:,[0,2]].values
Y = ANNDF.iloc[:, 1].values.reshape(-1,1)

xtrain, xtest, ytrain,  ytest = train_test_split(X, Y, train_size = 0.85)

scalerx = StandardScaler()
scalery = StandardScaler()

Xtrain = scalerx.fit_transform(xtrain)
Ytrain = scalery.fit_transform(ytrain)



mlp = MLPRegressor(max_iter=5000)

#-----------------------


# parameter grid, KFold cross-validation

kf = KFold(n_splits=10, shuffle=True)

parameters = {
    'hidden_layer_sizes': [(1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (8, 8), (4, 4, 4), (6, 6, 6), (8, 8, 8), (4, 4, 4, 4), (6, 6, 6, 6), (8, 8, 8, 8)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['adam']
}

#applying gridsearchcv, train the model

gsearchcv = GridSearchCV(estimator=mlp, param_grid=parameters, cv=kf)
gsearchcv.fit(Xtrain, Ytrain.ravel())

#define best score and parameters based on R²
bestac = gsearchcv.best_score_ * 100  # Convert to percentage
bestparam = gsearchcv.best_params_

print('Best Accuracy: {:.2f} %'.format(bestac))
print('Best Parameters:', bestparam)


#-------------

# Definir os intervalos para Re e Stdp
Re_vals = np.linspace(200, 8000, 100)  # Valores de Reynolds de 1000 a 8000
Stdp_vals = np.linspace(0.3326, 0.4331, 100)  # Valores de Stdp de 0.3326 a 0.4331

# Criar uma grade de Re e Stdp
Re_grid, Stdp_grid = np.meshgrid(Re_vals, Stdp_vals)
grid_points = np.c_[Re_grid.ravel(), Stdp_grid.ravel()]

# Prever os valores de 'f' usando a ANN nos pontos da grade
f_pred_scaled = gsearchcv.predict(scalerx.transform(grid_points))
f_pred = scalery.inverse_transform(f_pred_scaled.reshape(-1, 1))
f_pred_grid = f_pred.reshape(Re_grid.shape)

#---------------------------


input_data = np.array([[1843, 0.4331]])

# Transformar os dados de entrada usando o scaler treinado
input_data_scaled = scalerx.transform(input_data)

# Fazer a previsão com o modelo treinado
predicted_scaled = gsearchcv.predict(input_data_scaled)

# Inverter a transformação do output (scaling)
predicted = scalery.inverse_transform(predicted_scaled.reshape(-1, 1))

Xtest_scaled = scalerx.transform(xtest)

ypred = gsearchcv.predict(Xtest_scaled).reshape(-1,1)

ypred_or = scalery.inverse_transform(ypred)

# Mostrar o resultado
print('Predicted value for [2000, 0.24]:', predicted[0][0])


Best Accuracy: 89.95 %
Best Parameters: {'activation': 'tanh', 'hidden_layer_sizes': (8, 8, 8, 8), 'solver': 'adam'}
Predicted value for [2000, 0.24]: 0.6306582554250645


In [ ]:
from pysr import PySRRegressor


# Reajustar o modelo com PySR
model = PySRRegressor(
    niterations=20,  # Começar com menos iterações para evitar sobrecarga
    binary_operators=["+", "*", "-", "/"],
    unary_operators=["cos", "exp", "sin", "inv(x)=1/x"],
    extra_sympy_mappings={"inv": lambda x: 1/x},  # Definir a função `inv(x)`
    loss="loss(x, y) = (x - y)^2",  # Função de perda personalizada
    population_size=5,  # Tamanho da população reduzido para evitar sobrecarga
    progress=True,  # Mostrar o progresso
    verbosity=1,  # Nível de verbosidade
)

# Gerar previsões com o modelo já treinado para a superfície
predicted = gsearchcv.predict(Xtrain)
predicted = scalery.inverse_transform(predicted.reshape(-1, 1)).flatten()

# Ajustar o modelo PySR aos dados
# Usar os dados de treinamento após o escalonamento inverso
Re_values = xtrain[:, 0]
Stdp_values = xtrain[:, 1]

# Certificar que X_data e predicted tenham o mesmo número de amostras
X_data = np.column_stack((Re_values, Stdp_values))
model.fit(X_data, predicted)

# Mostrar as equações encontradas
print(model)

# Prever novos valores com a melhor equação simbólica encontrada
best_equation = model.predict(X_data)
print("Best symbolic equation prediction:", best_equation)

In [ ]:
def cor1(Re):
    f = 2.344 * np.power(Re,-0.081) #correlação kaixin yan  300 < RE <3100
    return f

def cor2(Re):
    f = 2.765 * np.power(Re,-0.166) #correlação kaixin yan  300 < RE <3100
    return f

Recor1 = np.linspace(300,3100,10)
stpkyan = [0.3745] * len(Recor1)

In [ ]:
import numpy as np
import plotly.graph_objs as go
import plotly.offline as pyo



# Definir os intervalos para Re e Stdp
Re_vals = np.linspace(200, 8000, 100)  # Valores de Reynolds de 1000 a 8000
Stdp_vals = np.linspace(0.3326, 0.4331, 100)  # Valores de Stdp de 0.3326 a 0.4331

ffc = 2.7753 + (-3.1833e-04)*Re_vals + (-7.3509e+00)*Stdp_vals + (1.1566e-08)*Re_vals^2 + (9.1489e-04)*Re_vals * Stdp_vals + (3.0968e+00)*Stdp_vals^2 + (-5.4668e-13)*Re_vals^3 + (-2.9617e-09)*Re_vals^2 * Stdp_vals + (-7.7045e-04)*Re_vals * Stdp_vals^2 + (6.7924e+00)*Stdp_vals^3

# Criar uma grade de Re e Stdp
Re_grid, Stdp_grid = np.meshgrid(Re_vals, Stdp_vals)
grid_points = np.c_[Re_grid.ravel(), Stdp_grid.ravel()]

# Prever os valores de 'f' usando a ANN nos pontos da grade
f_pred_scaled = gsearchcv.predict(scalerx.transform(grid_points))
f_pred = scalery.inverse_transform(f_pred_scaled.reshape(-1, 1))
f_pred_grid = f_pred.reshape(Re_grid.shape)

# Calcular a superfície usando a fórmula fornecida
f_formula = (4.3059 / np.log(Re_grid + 3.1918)) + ((0.30649 / Stdp_grid) - 0.80937)

# Criar o gráfico 3D para os resultados da ANN com contornos
surface_ann = go.Surface(
    x=Re_grid, y=Stdp_grid, z=f_pred_grid, 
    colorscale='Viridis',
    name='Superfície ANN',
    colorbar=dict(title='ANN f', len=0.5, x=0.9),
    contours={
        "z": {
            "show": True,
            "start": np.min(f_pred_grid),
            "end": np.max(f_pred_grid),
            "size": (np.max(f_pred_grid) - np.min(f_pred_grid)) / 20,  # Ajustar o espaçamento dos contornos
            "color": "black",
        }
    }
)

# Criar o gráfico 3D para a fórmula fornecida com contornos
surface_formula = go.Surface(
    x=Re_grid, y=Stdp_grid, z=f_formula, 
    colorscale='Plasma',
    name='Superfície Fórmula',
    opacity=0.7,
    colorbar=dict(title='Fórmula f', len=0.5, x=1.05),
    contours={
        "z": {
            "show": True,
            "start": np.min(f_formula),
            "end": np.max(f_formula),
            "size": (np.max(f_formula) - np.min(f_formula)) / 20,
            "color": "black",
        }
    }
)

# Scatter plot dos valores originais
scatter_cor = go.Scatter3d(
    x=Recor1,  # Valores de Reynolds
    y=stpkyan,  # Valores de Stdp
    z=cor2(Recor1),  # Valores de 'f'
    mode='markers',
    marker=dict(size=5, color='red'),
    name='Valores Originais'
)


# Scatter plot dos valores originais
scatter_data = go.Scatter3d(
    x=ANNDF.iloc[:, 0].values,  # Valores de Reynolds
    y=ANNDF.iloc[:, 2].values,  # Valores de Stdp
    z=ANNDF.iloc[:, 1].values,  # Valores de 'f'
    mode='markers',
    marker=dict(size=5, color='red'),
    name='Valores Originais'
)

# Definir o layout do gráfico
layout = go.Layout(
    title='Superfície de Resultados da ANN e da Fórmula com Valores Originais',
    scene=dict(
        xaxis=dict(title='Reynolds Number (Re)', backgroundcolor='white', gridcolor='lightgrey'),
        yaxis=dict(title='Stdp', backgroundcolor='white', gridcolor='lightgrey'),
        zaxis=dict(title='f', backgroundcolor='white', gridcolor='lightgrey'),
        bgcolor='white'
    ),
    paper_bgcolor='white',
    plot_bgcolor='white',
    showlegend=True
)

# Criar a figura com ambas as superfícies e o scatter plot
fig = go.Figure(data=[surface_ann ,surface_formula,scatter_cor], layout=layout)

# Mostrar o gráfico
pyo.plot(fig)


In [ ]:
import numpy as np
import plotly.graph_objs as go
import plotly.io as pio

# Definir os intervalos para Re e Stdp
Re_vals = np.linspace(1000, 8000, 100)  # Valores de Reynolds de 1000 a 8000
Stdp_vals = np.linspace(0.3326, 0.4331, 100)  # Valores de Stdp de 0.3326 a 0.4331

# Criar uma grade de Re e Stdp
Re_grid, Stdp_grid = np.meshgrid(Re_vals, Stdp_vals)

# Calcular a superfície usando a fórmula fornecida
f_formula = (4.3059 / np.log(Re_grid + 3.1918)) + ((0.30649 / Stdp_grid) - 0.80937)

# Criar o gráfico de contorno 2D para a fórmula fornecida
contour_formula = go.Contour(
    z=f_formula, 
    x=Re_vals, 
    y=Stdp_vals, 
    colorscale='Viridis', 
    name='Fórmula Contour',
    colorbar=dict(title='f')
)

# Definir o layout do gráfico
layout_formula = go.Layout(
    title='f Contour (Formula)',
    xaxis=dict(title='Reynolds Number (Re)'),
    yaxis=dict(title='Stdp'),
    width=800,  # Largura da imagem
    height=600  # Altura da imagem
)

# Criar a figura com o gráfico de contorno da fórmula
fig_formula = go.Figure(data=[contour_formula], layout=layout_formula)

# Mostrar o gráfico
pio.show(fig_formula)


In [ ]:
import numpy as np
import plotly.graph_objs as go
import plotly.io as pio

# Definir os intervalos para Re e Stdp
Re_vals = np.linspace(1000, 8000, 100)  # Valores de Reynolds de 1000 a 8000
Stdp_vals = np.linspace(0.3326, 0.4331, 100)  # Valores de Stdp de 0.3326 a 0.4331

# Criar uma grade de Re e Stdp
Re_grid, Stdp_grid = np.meshgrid(Re_vals, Stdp_vals)
grid_points = np.c_[Re_grid.ravel(), Stdp_grid.ravel()]

# Prever os valores de 'f' usando a ANN nos pontos da grade
f_pred_scaled = gsearchcv.predict(scalerx.transform(grid_points))
f_pred = scalery.inverse_transform(f_pred_scaled.reshape(-1, 1))
f_pred_grid = f_pred.reshape(Re_grid.shape)

# Criar o gráfico de contorno 2D para os resultados da ANN
contour_ann = go.Contour(
    z=f_pred_grid, 
    x=Re_vals, 
    y=Stdp_vals, 
    colorscale='Viridis', 
    name='ANN Contour',
    colorbar=dict(title='ANN f')
)

# Scatter plot dos valores originais
scatter_data = go.Scatter(
    x=ANNDF.iloc[:, 0].values,  # Valores de Reynolds
    y=ANNDF.iloc[:, 2].values,  # Valores de Stdp
    mode='markers',
    marker=dict(size=5, color='red'),
    name='Valores Originais'
)

# Definir o layout do gráfico
layout_ann = go.Layout(
    title='Contorno dos Resultados da ANN com Valores Originais',
    xaxis=dict(title='Reynolds Number (Re)'),
    yaxis=dict(title='Stdp'),
    width=800,  # Largura da imagem
    height=600  # Altura da imagem
)

# Criar a figura com o gráfico de contorno da ANN e o scatter plot dos valores originais
fig_ann = go.Figure(data=[contour_ann], layout=layout_ann)

# Mostrar o gráfico
pio.show(fig_ann)
